<h1>
Annotating Genomic Variants
</h1>
-   [Background](#background)
-   [Set Up](#setup)
-   [Exploring variants in the TRPV gene family](#TRPV)
    -   [Examine header data in a vcf file](#TRPV_header)
    -   [Convert gene symbols to gene ids](#TRPV_symbols)
    -   [Create gene ranges](#TRPV_ranges)
    -   [Extract variant subsets](#TRPV_subset)
    -   [Variant location in the gene model](#TRPV_locate)
    -   [Amino acid coding changes in non-synonymous variants](#TRPV_predict)
-   [Annotating with the ensemblVEP package](#vep)
-   [Exploring Package Content](#content)
-   [sessionInfo()](#session)

<h2 id="background">
Background
</h2>
The VariantAnnotation package has facilities for reading in all or subsets of Variant Call Format (VCF) files. These text files contain meta-information lines, a header line and data lines each containing information about a position in the genome. The format also may also contain genotype information on samples for each position. More on the file format can be found in the [VCF specs](http://samtools.github.io/hts-specs/VCFv4.2.pdf).

The 'locateVariants' function in the VariantAnnotation package identifies where a variant is located with respect to the gene model (e.g., exon, intron, splice site, etc.). The 'predictCoding' function reports the amino acid change for non-synonymous coding variants. Consequences of the coding changes can be investigated with the SIFT and PolyPhen database packages. We'll use these functions to learn about variants located on the TRPV gene on chromosome

<h2 id="setup">
Set Up
</h2>
This workflow requires several different Bioconductor packages. Usage of each will be described in detail in the following sections.

``` r
library(VariantAnnotation)
library(cgdv17)
library(org.Hs.eg.db)
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
library(BSgenome.Hsapiens.UCSC.hg19)
library(PolyPhen.Hsapiens.dbSNP131)
```

Use biocLite() to get the packages you don't have installed:

``` r
## try http:// if https:// URLs are not supported
source("https://bioconductor.org/biocLite.R")
biocLite("mypackage")
```

<h2 id="TRPV">
Exploring variants in the TRPV gene family
</h2>
This workflow focuses on variants located in the Transient Receptor Potential Vanilloid (TRPV) gene family on chromosome 17. Sample data are from the Bioconductor cgdv17 experimental data package which contains Complete Genomics Diversity panel data for chromosome 17 on 46 individuals. For more background on how these data were curated see the package vignette.

``` r
browseVignettes("cgdv17")
```

We use a VCF file from the package that is a subset of chromosome 17 for a single individual from the CEU population.

In [ ]:
library(VariantAnnotation)
library(cgdv17)
file <- system.file("vcf", "NA06985_17.vcf.gz", package = "cgdv17")

<h3 id="TRPV_header">
Examine header data in a vcf file
</h3>
To get an idea of what data are in the file we look at the header. scanVcfHeader() parses the file header into a VCFHeader object and the info() and geno() accessors extract field-specific data.

In [ ]:
hdr <- scanVcfHeader(file)
 
info(hdr) 
 
geno(hdr) 

Variants in the VCF have been aligned to NCBI genome build GRCh37:

In [ ]:
meta(hdr)$META

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h3 id="TRPV_symbols">
Convert gene symbols to gene ids
</h3>
Use the org.Hs.eg.db package to convert gene symbols to gene ids.

In [ ]:
## get entrez ids from gene symbols
library(org.Hs.eg.db)
genesym <- c("TRPV1", "TRPV2", "TRPV3")
geneid <- select(org.Hs.eg.db, keys=genesym, keytype="SYMBOL",
                 columns="ENTREZID")
geneid

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h3 id="TRPV_ranges">
Create gene ranges
</h3>
We use the hg19 known gene track from UCSC to identify the TRPV gene ranges. These ranges will eventually be used to extract variants from a regions in the VCF file.

Load the annotation package.

In [ ]:
library(TxDb.Hsapiens.UCSC.hg19.knownGene)
txdb <- TxDb.Hsapiens.UCSC.hg19.knownGene
txdb

Our VCF file was aligned to a genome from NCBI and the known gene track was from UCSC. These institutions have different naming conventions for the chromosomes. In order to use these two pieces of data in a matching or overlap operation the chromosome names (also called sesqlevels) need to match. We will modify the txdb to match the VCF file.

In [ ]:
txdb <- renameSeqlevels(txdb, gsub("chr", "", seqlevels(txdb)))
txdb <- keepSeqlevels(txdb, "17")

Create a list of transcripts by gene:

In [ ]:
txbygene = transcriptsBy(txdb, "gene")

Create the gene ranges for the TRPV genes

In [ ]:
gnrng <- unlist(range(txbygene[geneid$ENTREZID]), use.names=FALSE)
names(gnrng) <- geneid$SYMBOL

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h3 id="TRPV_subset">
Extract variant subsets
</h3>
A ScanVcfParam object is used to retrieve data subsets. This object can specify genomic coordinates (ranges) or individual VCF elements. Extractions of ranges (vs fields) requires a tabix index. See ?indexTabix for details.

In [ ]:
param <- ScanVcfParam(which = gnrng, info = "DP", geno = c("GT", "cPd"))
param
 
## Extract the TRPV ranges from the VCF file 
vcf <- readVcf(file, "hg19", param)
## Inspect the VCF object with the 'fixed', 'info' and 'geno' accessors
vcf
 
head(fixed(vcf))

geno(vcf)

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h3 id="TRPV_locate">
Variant location in the gene model
</h3>
The locateVariants function identifies where a variant falls with respect to gene structure, e.g., exon, utr, splice site, etc. We use the gene model from the TxDb.Hsapiens.UCSC.hg19.knownGene package loaded eariler.

``` r
## Use the 'region' argument to define the region
## of interest. See ?locateVariants for details.
cds <- locateVariants(vcf, txdb, CodingVariants())
five <- locateVariants(vcf, txdb, FiveUTRVariants())
splice <- locateVariants(vcf, txdb, SpliceSiteVariants())
intron <- locateVariants(vcf, txdb, IntronVariants())
```

In [ ]:
all <- locateVariants(vcf, txdb, AllVariants())

Each row in cds represents a variant-transcript match so multiple rows per variant are possible. If we are interested in gene-centric questions the data can be summarized by gene regardless of transcript.

In [ ]:
## Did any variants match more than one gene?
table(sapply(split(mcols(all)$GENEID, mcols(all)$QUERYID), 
      function(x) length(unique(x)) > 1))
 
## Summarize the number of variants by gene:
idx <- sapply(split(mcols(all)$QUERYID, mcols(all)$GENEID), unique)
sapply(idx, length)
 
## Summarize variant location by gene:
sapply(names(idx), 
    function(nm) {
        d <- all[mcols(all)$GENEID %in% nm, c("QUERYID", "LOCATION")]
        table(mcols(d)$LOCATION[duplicated(d) == FALSE])
    })

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h3 id="TRPV_predict">
Amino acid coding changes in non-synonymous variants
</h3>
Amino acid coding for non-synonymous variants can be computed with the function predictCoding. The BSgenome.Hsapiens.UCSC.hg19 package is used as the source of the reference alleles. Variant alleles are provided by the user.

In [ ]:
library(BSgenome.Hsapiens.UCSC.hg19)
seqlevelsStyle(vcf) <- "UCSC"
seqlevelsStyle(txdb) <- "UCSC"
aa <- predictCoding(vcf, txdb, Hsapiens)

predictCoding returns results for coding variants only. As with locateVariants, the output has one row per variant-transcript match so multiple rows per variant are possible.

In [ ]:
## Did any variants match more than one gene?
table(sapply(split(mcols(aa)$GENEID, mcols(aa)$QUERYID), 
        function(x) length(unique(x)) > 1))

## Summarize the number of variants by gene:
idx <- sapply(split(mcols(aa)$QUERYID, mcols(aa)$GENEID, drop=TRUE), unique)
sapply(idx, length)

## Summarize variant consequence by gene:
sapply(names(idx), 
       function(nm) {
           d <- aa[mcols(aa)$GENEID %in% nm, c("QUERYID","CONSEQUENCE")]
           table(mcols(d)$CONSEQUENCE[duplicated(d) == FALSE])
       })

The variants 'not translated' are explained by the warnings thrown when predictCoding was called. Variants that have a missing varAllele or have an 'N' in the varAllele are not translated. If the varAllele substitution had resulted in a frameshift the consequence would be 'frameshift'. See ?predictCoding for details.

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h2 id="vep">
Annotating with the ensemblVEP package
</h2>
The ensemblVEP package provides access to the online Ensembl Variant Effect Predictor (VEP tool). The VEP tool ouputs predictions of functional consequences of known and unknown variants as reported by Sequence Ontology or Ensembl. Regulatory region consequences, HGNC, Ensembl protein identifiers, HGVS, co-located variants are optional outputs. ensemblVEP() accepts the name of a VCF file and returns a VCF on disk or GRanges in the R workspace.

Load ensemblVEP:

In [ ]:
library(ensemblVEP)

The 'file' argument to ensemblVEP must be a vcf file on disk. We'll write out the VCF object with the TRPV variants and submit that to ensemblVEP.

Write the VCF object to a vcf file in tempfile():

In [ ]:
dest <- tempfile()
writeVcf(vcf, dest)

Call ensemblVEP with the file containing only the TRPV variants and the custom VEPParam object:

``` r
gr <- ensemblVEP(file = dest)
```

``` r
2015-06-09 08:50:22 - Starting...
2015-06-09 08:50:24 - Detected format of input file as vcf
2015-06-09 08:50:24 - Read 277 variants into buffer
2015-06-09 08:50:24 - Reading transcript data from cache and/or database
[================================================================================================================================================================]  [ 100% ]
2015-06-09 08:54:29 - Retrieved 55 transcripts (0 mem, 0 cached, 55 DB, 0 duplicates)
2015-06-09 08:54:29 - Analyzing chromosome 17
2015-06-09 08:54:29 - Analyzing variants
[================================================================================================================================================================]  [ 100% ]
2015-06-09 08:54:29 - Calculating consequences
[================================================================================================================================================================]  [ 100% ]
2015-06-09 08:54:30 - Processed 277 total variants (1 vars/sec, 1 vars/sec total)
2015-06-09 08:54:30 - Wrote stats summary to /tmp/RtmppUYah1/file339f64bf5700_summary.html
2015-06-09 08:54:30 - Finished!
```

``` r
head(gr, 3)
```

``` r
GRanges object with 3 ranges and 22 metadata columns:
           seqnames             ranges strand |   Allele
              <Rle>          <IRanges>  <Rle> | <factor>
  rs402369    chr17 [3469401, 3469401]      * |        G
  rs402369    chr17 [3469401, 3469401]      * |        G
  rs402369    chr17 [3469401, 3469401]      * |        G
                                    Consequence   IMPACT   SYMBOL
                                       <factor> <factor> <factor>
  rs402369 splice_region_variant&intron_variant      LOW  SPATA22
  rs402369 splice_region_variant&intron_variant      LOW  SPATA22
  rs402369                upstream_gene_variant MODIFIER     ASPA
                      Gene Feature_type         Feature        BIOTYPE     EXON
                  <factor>     <factor>        <factor>       <factor> <factor>
  rs402369 ENSG00000141255   Transcript ENST00000397168 protein_coding     <NA>
  rs402369 ENSG00000141255   Transcript ENST00000355380 protein_coding     <NA>
  rs402369 ENSG00000108381   Transcript ENST00000456349 protein_coding     <NA>
             INTRON    HGVSc    HGVSp cDNA_position CDS_position
           <factor> <factor> <factor>      <factor>     <factor>
  rs402369      1/8     <NA>     <NA>          <NA>         <NA>
  rs402369      1/7     <NA>     <NA>          <NA>         <NA>
  rs402369     <NA>     <NA>     <NA>          <NA>         <NA>
           Protein_position Amino_acids   Codons Existing_variation DISTANCE
                   <factor>    <factor> <factor>           <factor> <factor>
  rs402369             <NA>        <NA>     <NA>               <NA>     <NA>
  rs402369             <NA>        <NA>     <NA>               <NA>     <NA>
  rs402369             <NA>        <NA>     <NA>               <NA>     4652
             STRAND SYMBOL_SOURCE    HGNC_ID
           <factor>      <factor>   <factor>
  rs402369       -1          HGNC HGNC:30705
  rs402369       -1          HGNC HGNC:30705
  rs402369        1          HGNC   HGNC:756
  -------
  seqinfo: 25 sequences from  genome; no seqlengths
```

Data from the VEP tool is returned as metadata columns in the GRanges ('gr' object). You can further control what fields are returned by setting the runtime options in the VEPParam.

In [ ]:
VEPParam()

Each of the options, e.g., 'basic', 'inut', 'cache' etc. has multiple fields that can be set.

In [ ]:
basic(VEPParam())

For more details on runtime options see the ?VEPParam man page and the [Ensembl VEP web site](http://uswest.ensembl.org/info/docs/tools/vep/script/vep_options.html).

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h2 id="content">
Exploring Package Content
</h2>
Packages have extensive help pages, and include vignettes highlighting common use cases. The help pages and vignettes are available from within R. After loading a package, use syntax like

In [ ]:
help(package="VariantAnnotation")
?predictCoding

to obtain an overview of help on the `VariantAnnotation` package, and the `predictCoding` function. View the package vignette with

``` r
browseVignettes(package="VariantAnnotation")
```

To view vignettes providing a more comprehensive introduction to package functionality use

``` r
help.start()
```

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>
<h2 id="session">
sessionInfo()
</h2>

In [ ]:
sessionInfo()

<p class="back_to_top">
\[ <a href="#top">Back to top</a> \]
</p>